# Get Set Up

## Import Libraries

In [0]:
# Pandas provides an extremely useful data structure
import pandas as pd

# RE provides regular expression pattern matching
import re

# datetime provides a datetime object class and conversion utilities
from datetime import datetime

# Web file access
from urllib.request import urlopen




## Define Some Functions

In [0]:
def log_ReadFile(logfile):
  with open(logfile) as fh:
    loglines = fh.readlines()
  loglines = [line.strip() for line in loglines]
  return loglines


def log_ReadURL(logfile):
  loglines = urlopen(logfile).readlines()
  loglines = [line.decode().strip() for line in loglines]
  return loglines

# Instructions

The Data Basics lab presents the process of preparing and manipulating data.  Using two datasets, the differences between structured and unstructured data are highlighted.  The basic operations of relational algebra are explored using the Pandas DataFrame Class.  An example of text parsing using regular expressions is also provided.

* Read through the explanations and examples in each section.

* Based on the examples, complete the exercises labeled "Try it yourself.""

* This lab is intended to develop techniques that will be used throughout the Exploratory Data Analysis lab later on.

**To begin, choose "Runtime -> Run All" from the menu options.**

# Structured Data: Threat Indicators

## Prepare Data

Preparing structured data is relatively easy, as the structure can be relied on to simplicy parsing.  Here, we load a prepared collection of malicious IP address indicators into a Pandas dataframe.

In [0]:
# URL of source data file
intel_url = "https://raw.githubusercontent.com/urbansec/ds101/master/av_ip_reputation_2019-04-07.csv"

# Column names for data
intel_cols = ['ip', 'risk', 'reliability', 'activity', 'country', 'city', 'lat_lon', 'unknown']


# Read intel into dataframe
intel_df = pd.read_csv(intel_url, sep='#', header=None, names=intel_cols)

## Examine Results

The ".head()" method can be used to preview the first few lines of a data frame.

In [5]:
intel_df.head(5)

,ip,risk,reliability,activity,country,city,lat_lon,unknown
0,46.246.124.162,4,2,Malicious Host,SE,NaN,"59.3246994019,18.0559997559",3
1,23.94.213.6,4,2,Malicious Host,US,Los Angeles,"34.0544013977,-118.244003296",3
2,23.116.185.114,4,2,Malicious Host,US,NaN,"37.7509994507,-97.8219985962",3
3,23.30.95.53,4,2,Malicious Host,US,NaN,"37.7509994507,-97.8219985962",3
4,14.58.252.223,4,2,Malicious Host,KR,NaN,"37.5111999512,126.974098206",3


# Unstructured Data: Web Error Logs

## Prepare Data

Preparing unstructured data can be difficult.  By it's nature, it is often irregular and you will be responsible for determining reliable patterns that can be used duing parsing.  This process is showen below for a sample of web server error logs.  Machine logs, while unstructured, tend to be more regular in format than data generated by people.  You will see that we are able to parse these logs into a structured form via automation we write.

### Download the file to a list

In [0]:
# URL of source data file
error_url = "https://raw.githubusercontent.com/urbansec/ds101/master/error.log.2019-03-22"

# Read log files into lists
error_logs = log_ReadURL(error_url)

### Examine the downloaded content

In [7]:
# check our data type
type(error_logs)

list

In [8]:
# check data type of lists members
type(error_logs[0])

str

In [9]:
# display first 5 lines in list
display(error_logs[:5])

['[Fri Mar 22 01:58:55 2019] [error] [client 54.36.148.18] File does not exist: /home/sooshie/secrepo.com/self.logs/error.log.2016-04-07.gz',
 '[Fri Mar 22 02:04:26 2019] [error] [client 54.36.148.62] File does not exist: /home/sooshie/secrepo.com/self.logs/access.log.2016-10-30.gz',
 '[Fri Mar 22 02:08:27 2019] [error] [client 27.255.4.117] ModSecurity: Access denied with code 418 (phase 1). Pattern match "^Mozilla/5.0 \\\\\\\\(Windows NT 6.1; WOW64; rv:40.0\\\\\\\\) Gecko/20100101 Firefox/40.1$" at REQUEST_HEADERS:User-Agent. [file "/dh/apache2/template/etc/mod_sec2/99_dreamhost_rules.conf"] [line "345"] [id "1990098"] [msg "Malicious Bot UA"] [hostname "www.secrepo.com"] [uri "/wp-login.php"] [unique_id "XJSmC9BxoAQAACFcAa8AAAAE"]',
 '[Fri Mar 22 02:08:28 2019] [error] [client 27.255.4.117] ModSecurity: Access denied with code 418 (phase 1). Pattern match "^Mozilla/5.0 \\\\\\\\(Windows NT 6.1; WOW64; rv:40.0\\\\\\\\) Gecko/20100101 Firefox/40.1$" at REQUEST_HEADERS:User-Agent. [file

### Parse the data

Regular expressions can be used to flexibly parse each line of the log file into separate fields based on a pattern.
For more info on regular expressions, check out the re library documentation
https://docs.python.org/3/library/re.html

In [0]:
# define a regex pattern to parse lines into fields
# sample line:
# '[Fri Mar 22 02:13:49 2019] [error] [client 54.36.149.5] File does not exist: /home/sooshie/secrepo.com/self.logs/access.log.2015-09-07.gz']
web_error_pattern = re.compile('\[(?P<date>[^\]]+)\]'
                                '\s+\[(?P<msg_type>[^\]]+)\]'
                                '\s+\[client\s+(?P<client_ip>[^\]]+)\]'
                                '\s+(?P<error_type>[^:]+):'
                                '\s+(?P<error_message>.*)')


Once the regex parses each field correctly, use it to parse each line.  Add functionality to display lines that are not parsed correctly.

In [0]:
# create an empty lists to contain results
error_logs_parsed = []
parse_fails = []

# loop through input
for line in error_logs:
  match = web_error_pattern.match(line)
  if match:
    # parse fields into matrix
    error_logs_parsed.append([match.group('date'), 
                               match.group('msg_type'),
                               match.group('client_ip'), 
                               match.group('error_type'), 
                               match.group('error_message')])
  else:
    # save all failed attempts to a separate list
    parse_fails.append(line)

Examine parsed fields, and check for lines that were not parsed successfully.

In [12]:
# display first parsed line
error_logs_parsed[0]

['Fri Mar 22 01:58:55 2019',
 'error',
 '54.36.148.18',
 'File does not exist',
 '/home/sooshie/secrepo.com/self.logs/error.log.2016-04-07.gz']

In [13]:
# display any lines for which parsing failed
# should be empty
parse_fails

[]

### Convert to DataFrame

Examine the contents of our parsed data structure.  It's handy, but could be more efficient.  Convert to a Pandas dataframe so it's easier to work with.

In [0]:
# define list of columns to use
error_column_list = ['date', 'msg_type', 'client_ip', 'error_type', 'error_message']

# convert to Pandas dataframe and display it
error_logs_df = pd.DataFrame.from_records(error_logs_parsed, columns=error_column_list)

## Examine Results

In [15]:
error_logs_df.head()

,date,msg_type,client_ip,error_type,error_message
0,Fri Mar 22 01:58:55 2019,error,54.36.148.18,File does not exist,/home/sooshie/secrepo.com/self.logs/error.log....
1,Fri Mar 22 02:04:26 2019,error,54.36.148.62,File does not exist,/home/sooshie/secrepo.com/self.logs/access.log...
2,Fri Mar 22 02:08:27 2019,error,27.255.4.117,ModSecurity,Access denied with code 418 (phase 1). Pattern...
3,Fri Mar 22 02:08:28 2019,error,27.255.4.117,ModSecurity,Access denied with code 418 (phase 1). Pattern...
4,Fri Mar 22 02:13:49 2019,error,54.36.149.5,File does not exist,/home/sooshie/secrepo.com/self.logs/access.log...


Check out what data type each column is defined as using DataFrame.dtypes attribute

In [16]:
error_logs_df.dtypes

date             object
msg_type         object
client_ip        object
error_type       object
error_message    object
dtype: object

# Basic Data Operations

## Subset / Select

Select a subset of columns

In [17]:
error_logs_df[['client_ip', 'error_message']].head()

,client_ip,error_message
0,54.36.148.18,/home/sooshie/secrepo.com/self.logs/error.log....
1,54.36.148.62,/home/sooshie/secrepo.com/self.logs/access.log...
2,27.255.4.117,Access denied with code 418 (phase 1). Pattern...
3,27.255.4.117,Access denied with code 418 (phase 1). Pattern...
4,54.36.149.5,/home/sooshie/secrepo.com/self.logs/access.log...


### Try it yourself:

Modify the above command to display only the "date", "msg_type", and "error_type" columns.

In [0]:
# Place your code here


Select a subset of rows

In [18]:
error_logs_df.iloc[10:15,]

,date,msg_type,client_ip,error_type,error_message
10,Fri Mar 22 02:35:30 2019,error,123.129.224.7,File does not exist,"/home/sooshie/secrepo.com/utility, referer: ht..."
11,Fri Mar 22 02:35:31 2019,error,123.129.224.7,File does not exist,"/home/sooshie/secrepo.com/uploads, referer: ht..."
12,Fri Mar 22 02:35:31 2019,error,123.129.224.7,File does not exist,"/home/sooshie/secrepo.com/uploads, referer: ht..."
13,Fri Mar 22 02:45:50 2019,error,92.241.103.84,ModSecurity,Access denied with code 418 (phase 1). Pattern...
14,Fri Mar 22 02:45:50 2019,error,92.241.103.84,ModSecurity,Access denied with code 418 (phase 1). Pattern...


## Add a column

Blah blah blah

In [19]:
error_logs_df['web_server'] = "www.secrepo.com"
error_logs_df.head()


,date,msg_type,client_ip,error_type,error_message,web_server
0,Fri Mar 22 01:58:55 2019,error,54.36.148.18,File does not exist,/home/sooshie/secrepo.com/self.logs/error.log....,www.secrepo.com
1,Fri Mar 22 02:04:26 2019,error,54.36.148.62,File does not exist,/home/sooshie/secrepo.com/self.logs/access.log...,www.secrepo.com
2,Fri Mar 22 02:08:27 2019,error,27.255.4.117,ModSecurity,Access denied with code 418 (phase 1). Pattern...,www.secrepo.com
3,Fri Mar 22 02:08:28 2019,error,27.255.4.117,ModSecurity,Access denied with code 418 (phase 1). Pattern...,www.secrepo.com
4,Fri Mar 22 02:13:49 2019,error,54.36.149.5,File does not exist,/home/sooshie/secrepo.com/self.logs/access.log...,www.secrepo.com


## Add a row

Blah blah blah

In [20]:
# define new row to add as a list
new_row = ['2019-03-22 02:13:49', 'test', '201.55.86.182', 'New row added', 'We added a test row.  Did it work?', 'www.secrepo.com']

# column names are critical, so define here as a safeguard
error_column_list = ['date', 'msg_type', 'client_ip', 'error_type', 'error_message', 'web_server']

# convert new row to data frame and append to existing data frame
error_logs_df = error_logs_df.append(pd.DataFrame([new_row], columns=error_column_list), ignore_index=True)
error_logs_df.tail()

,date,msg_type,client_ip,error_type,error_message,web_server
272,Sat Mar 23 02:00:53 2019,error,122.114.72.221,File does not exist,"/home/sooshie/secrepo.com/plus, referer: http:...",www.secrepo.com
273,Sat Mar 23 02:02:36 2019,error,54.36.148.99,File does not exist,/home/sooshie/secrepo.com/self.logs/error.log....,www.secrepo.com
274,Sat Mar 23 02:05:54 2019,error,54.36.149.96,File does not exist,/home/sooshie/secrepo.com/self.logs/access.log...,www.secrepo.com
275,Sat Mar 23 02:08:08 2019,error,54.36.148.231,File does not exist,/home/sooshie/secrepo.com/self.logs/access.log...,www.secrepo.com
276,2019-03-22 02:13:49,test,201.55.86.182,New row added,We added a test row. Did it work?,www.secrepo.com


## Rename a column

Columns can be renamed using the ".rename()" method.  This is often useful when preparing data.  When used in a database, this is part of modifying the schema.

In [21]:
error_logs_df.rename(columns={'web_server':'server_name'}, inplace=True)
error_logs_df.head()

,date,msg_type,client_ip,error_type,error_message,server_name
0,Fri Mar 22 01:58:55 2019,error,54.36.148.18,File does not exist,/home/sooshie/secrepo.com/self.logs/error.log....,www.secrepo.com
1,Fri Mar 22 02:04:26 2019,error,54.36.148.62,File does not exist,/home/sooshie/secrepo.com/self.logs/access.log...,www.secrepo.com
2,Fri Mar 22 02:08:27 2019,error,27.255.4.117,ModSecurity,Access denied with code 418 (phase 1). Pattern...,www.secrepo.com
3,Fri Mar 22 02:08:28 2019,error,27.255.4.117,ModSecurity,Access denied with code 418 (phase 1). Pattern...,www.secrepo.com
4,Fri Mar 22 02:13:49 2019,error,54.36.149.5,File does not exist,/home/sooshie/secrepo.com/self.logs/access.log...,www.secrepo.com


## Transform a column

Transforming an existing column is similar to adding a new column.  In this case, the old column is replaced with a new column of the same name, containing the transformed values.

This could be useful for many things including rounding numerical values, substituting string values with shortened or normalized values, or replacing values with a simple Boolean.

Here, we modify the "date" column to use a python datetime object instead of a string.  This could be seen as a schema modification as well, as we are changing the data type of the column.

In [22]:
error_logs_df['date'] = pd.to_datetime(error_logs_df['date'])
error_logs_df.head()

,date,msg_type,client_ip,error_type,error_message,server_name
0,2019-03-22 01:58:55,error,54.36.148.18,File does not exist,/home/sooshie/secrepo.com/self.logs/error.log....,www.secrepo.com
1,2019-03-22 02:04:26,error,54.36.148.62,File does not exist,/home/sooshie/secrepo.com/self.logs/access.log...,www.secrepo.com
2,2019-03-22 02:08:27,error,27.255.4.117,ModSecurity,Access denied with code 418 (phase 1). Pattern...,www.secrepo.com
3,2019-03-22 02:08:28,error,27.255.4.117,ModSecurity,Access denied with code 418 (phase 1). Pattern...,www.secrepo.com
4,2019-03-22 02:13:49,error,54.36.149.5,File does not exist,/home/sooshie/secrepo.com/self.logs/access.log...,www.secrepo.com


# More Interesting Stuff

## Select Where (condition is met)

Using criteria to select only certain rows allows you to search  and filter your data set.  This is one of the most useful basic data functions.

You can search for a specific IP

In [23]:
# 54.36.150.75
error_logs_df.loc[error_logs_df['client_ip'] == '54.36.150.75',]

,date,msg_type,client_ip,error_type,error_message,server_name
201,2019-03-22 16:06:14,error,54.36.150.75,File does not exist,/home/sooshie/secrepo.com/self.logs/error.log....,www.secrepo.com


You can search for text within fields in a similar way (it's not necessary to do an exact '==' match).  We could also search for specific IP subnets or other factors.  We can even combine multiple criteria.

Here we search for errors containing the word "admin" from the subnet "222.186.x.x".  

In [24]:
error_logs_df[error_logs_df.error_message.str.contains("admin") & error_logs_df.client_ip.str.startswith("222.186")]

,date,msg_type,client_ip,error_type,error_message,server_name
36,2019-03-22 03:55:33,error,222.186.160.61,File does not exist,"/home/sooshie/secrepo.com/admin, referer: http...",www.secrepo.com
54,2019-03-22 04:12:45,error,222.186.59.44,File does not exist,"/home/sooshie/secrepo.com/admin, referer: http...",www.secrepo.com
153,2019-03-22 13:05:36,error,222.186.57.109,File does not exist,"/home/sooshie/secrepo.com/admin, referer: http...",www.secrepo.com


### Try it yourself:

Modify the search above to return rows with the text "Access denied" in the "error_message" field.  Don't include any criteria for the "client_ip" field.

Note: this is a case sensitive string match, so make sure the "A" is capitalized.

In [0]:
# Put your code here


## Join On (index field)

Join operations allow you to combine two data sets based on a shared piece of information.  In data science, this is often used to add context to the base data set.  Here, we can join our threat indicator data with out error log data to see if any errors were generated by known bad IPs.

In [26]:
joined_data = error_logs_df.set_index('client_ip').join(intel_df.set_index('ip'))
joined_data.head()

,date,msg_type,error_type,error_message,server_name,risk,reliability,activity,country,city,lat_lon,unknown
103.75.46.114,2019-03-22 14:47:21,error,File does not exist,/home/sooshie/secrepo.com/images,www.secrepo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103.75.46.114,2019-03-22 14:47:21,error,File does not exist,/home/sooshie/secrepo.com/config,www.secrepo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103.75.46.114,2019-03-22 14:47:22,error,File does not exist,/home/sooshie/secrepo.com/SiteFiles,www.secrepo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103.75.46.114,2019-03-22 14:47:22,error,File does not exist,/home/sooshie/secrepo.com/js,www.secrepo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103.75.46.114,2019-03-22 14:47:22,error,File does not exist,/home/sooshie/secrepo.com/siteserver,www.secrepo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The columns from the threat indicator data frame appear with those of the error log data frame.  For IPs that did not match a malicious IP, the values of the new columns are set to "NaN", which is like a null value.

We can search for values that are not NaN to find any matches that exist.  In this case, only our test row should match.

In [27]:
joined_data.loc[pd.notna(joined_data['activity']),]

,date,msg_type,error_type,error_message,server_name,risk,reliability,activity,country,city,lat_lon,unknown
201.55.86.182,2019-03-22 02:13:49,test,New row added,We added a test row. Did it work?,www.secrepo.com,4.0,2.0,Malicious Host,NaN,NaN,"-23.5,-47.4595985413",3
